# C9_LPF_int

In [1]:
import numpy as np
import pandas as pd
import os
from scipy.signal import butter, lfilter, freqz

In [2]:
def butter_lowpass(cutoff, fs, order=4):
    nyq = 0.5 * fs
    normal_cutoff = cutoff / nyq
    b, a = butter(order, normal_cutoff, btype='low', analog=False)
    return b, a

def butter_lowpass_filter(data, cutoff, fs, order=4):
    b, a = butter_lowpass(cutoff, fs, order=order)
    x = np.trunc(np.array(data, dtype=np.float64)).astype(np.int64)
    y = lfilter(b, a, x, axis=-1, zi=None)
    return y

# Setting standard filter requirements.
order = 4
fs = 200
cutoff = 5

b, a = butter_lowpass(cutoff, fs, order)

# Creating the data for filteration
T = 200         # value taken in seconds
n = int(T * fs) # indicates total samples
t = np.linspace(0, T, n, endpoint=False)

# ADL path

In [3]:
paths = []

for i in range(1, 20):
    root, file = os.path.split("../../Dataset/SA01/D01_SA01_R01.csv")
    if i < 10:
        file = file[:2] + str(i) + file[3:]
        tmp = os.path.join(root, file)
        paths.append(tmp)
        if i > 4:
            for j in range(2, 6):
                file_new = file[:-5] + str(j) + file[-4:]
                tmp = os.path.join(root, file_new)
                paths.append(tmp)
    if i >= 10:
        file = file[:1] + str(i) + file[3:]
        for j in range(1, 6):
            file = file[:-5] + str(j) + file[-4:]
            tmp = os.path.join(root, file)
            paths.append(tmp)

paths[4:14] = [ '../../Dataset/SA01/D05_SA01_R01_1.csv',
                '../../Dataset/SA01/D05_SA01_R01_2.csv',
                '../../Dataset/SA01/D05_SA01_R02_1.csv',
                '../../Dataset/SA01/D05_SA01_R02_2.csv',
                '../../Dataset/SA01/D05_SA01_R03_1.csv',
                '../../Dataset/SA01/D05_SA01_R03_2.csv',
                '../../Dataset/SA01/D05_SA01_R04_1.csv',
                '../../Dataset/SA01/D05_SA01_R04_2.csv',
                '../../Dataset/SA01/D05_SA01_R05_1.csv',
                '../../Dataset/SA01/D05_SA01_R05_2.csv',
                '../../Dataset/SA01/D06_SA01_R01_1.csv',
                '../../Dataset/SA01/D06_SA01_R01_2.csv',
                '../../Dataset/SA01/D06_SA01_R02_1.csv',
                '../../Dataset/SA01/D06_SA01_R02_2.csv',
                '../../Dataset/SA01/D06_SA01_R03_1.csv',
                '../../Dataset/SA01/D06_SA01_R03_2.csv',
                '../../Dataset/SA01/D06_SA01_R04_1.csv',
                '../../Dataset/SA01/D06_SA01_R04_2.csv',
                '../../Dataset/SA01/D06_SA01_R05_1.csv',
                '../../Dataset/SA01/D06_SA01_R05_2.csv']

paths[-15:-10] =['../../Dataset/SA01/D17_SA01_R01_1.csv',
                 '../../Dataset/SA01/D17_SA01_R01_2.csv',
                 '../../Dataset/SA01/D17_SA01_R02_1.csv',
                 '../../Dataset/SA01/D17_SA01_R02_2.csv',
                 '../../Dataset/SA01/D17_SA01_R03_1.csv',
                 '../../Dataset/SA01/D17_SA01_R03_2.csv',
                 '../../Dataset/SA01/D17_SA01_R04_1.csv',
                 '../../Dataset/SA01/D17_SA01_R04_2.csv',
                 '../../Dataset/SA01/D17_SA01_R05_1.csv',
                 '../../Dataset/SA01/D17_SA01_R05_2.csv']


subject_list = ['SA01', 'SA02', 'SA03', 'SA04', 'SA05',
                'SA06', 'SA07', 'SA08', 'SA09', 'SA10',
                'SA11', 'SA12', 'SA13', 'SA14', 'SA15',
                'SA16', 'SA17', 'SA18', 'SA19', 'SA20',
                'SA21', 'SA22', 'SA23',
                'SE01', 'SE02', 'SE03', 'SE04', 'SE05',
                'SE06', 'SE07', 'SE08', 'SE09', 'SE10',
                'SE11', 'SE12', 'SE13', 'SE14', 'SE15']


total_adl_path = []

for path in paths:
    for subject_name in subject_list:
        path_rename = path.replace('SA01', subject_name)
        total_adl_path.append(path_rename)

len(total_adl_path)

3572

# ADL data 전처리

In [4]:
adl_list = []

for path in total_adl_path:
    try:
        data = pd.read_csv(path, header=None)
        data = data.iloc[:, :3]
        data = np.abs(data)
                    
        max_value1 = data[0][200:].max()
        max_value2 = data[1][200:].max()
        max_value3 = data[2][200:].max()
        
        max_idx1 = data[ data[0] == max_value1 ].index[0]
        max_idx2 = data[ data[1] == max_value2 ].index[0]
        max_idx3 = data[ data[2] == max_value3 ].index[0]
        
        min_idx = min(max_idx1, max_idx2, max_idx3)
        sliced_df = data[min_idx - 200 : min_idx - 40]
        
        col_1_lpf = butter_lowpass_filter(sliced_df[0].to_numpy(), cutoff, fs, order).astype('int64')
        col_2_lpf = butter_lowpass_filter(sliced_df[1].to_numpy(), cutoff, fs, order).astype('int64')
        col_3_lpf = butter_lowpass_filter(sliced_df[2].to_numpy(), cutoff, fs, order).astype('int64')
    
    
        col_1_std = np.sqrt(np.absolute(np.mean(sliced_df[0]) - sliced_df[0])**2 / len(sliced_df[0]))
        col_2_std = np.sqrt(np.absolute(np.mean(sliced_df[1]) - sliced_df[1])**2 / len(sliced_df[1]))
        col_3_std = np.sqrt(np.absolute(np.mean(sliced_df[2]) - sliced_df[2])**2 / len(sliced_df[2]))

        col_1 = col_1_std*col_1_std
        col_2 = col_2_std*col_2_std
        col_3 = col_3_std*col_3_std
        
        C_nine = np.sqrt(col_1 + col_2 + col_3).tolist()
        adl_list.append(C_nine)
        # adl_first = pd.concat([list_C_nine, adl_first], axis=0, ignore_index=True)
         
    except:
        pass
    

adl_df = pd.DataFrame(adl_list)
print(adl_df.shape)
row, col = adl_df.shape


target_col = row * [0]
adl_df['target'] = target_col

print(adl_df.shape)

(3209, 160)
(3209, 161)


# ADL data 저장

In [5]:
final = pd.DataFrame(adl_df)
final = final.dropna(axis=0)
final.dropna()
final = final.astype(int)

final.to_csv('../result/a_c9_LPF_int.csv', index=False)

# Fall path

In [6]:
fall_list = []

for i in range(1, 16):
    root, file = os.path.split("../../Dataset/SA01/F01_SA01_R01.csv")
    if i < 10:
        file = file[:2] + str(i) + file[3:]
        tmp = os.path.join(root, file)
        for j in range(1, 6):
            file_new = file[:-5] + str(j) + file[-4:]
            tmp = os.path.join(root, file_new)
            fall_list.append(tmp)
    if i >= 10:
        file = file[:1] + str(i) + file[3:]
        tmp = os.path.join(root, file)
        for j in range(1, 6):
            file_new = file[:-5] + str(j) + file[-4:]
            tmp = os.path.join(root, file_new)
            fall_list.append(tmp)
            
print(fall_list)

subject_list = ['SA01', 'SA02', 'SA03', 'SA04', 'SA05',
                'SA06', 'SA07', 'SA08', 'SA09', 'SA10',
                'SA11', 'SA12', 'SA13', 'SA14', 'SA15',
                'SA16', 'SA17', 'SA18', 'SA19', 'SA20',
                'SA21', 'SA22', 'SA23', 'SE06']

total_fall_path = []

for path in fall_list:
    for subject_name in subject_list:
        path_rename = path.replace('SA01', subject_name)
        total_fall_path.append(path_rename)

total_fall_path

['../../Dataset/SA01/F01_SA01_R01.csv', '../../Dataset/SA01/F01_SA01_R02.csv', '../../Dataset/SA01/F01_SA01_R03.csv', '../../Dataset/SA01/F01_SA01_R04.csv', '../../Dataset/SA01/F01_SA01_R05.csv', '../../Dataset/SA01/F02_SA01_R01.csv', '../../Dataset/SA01/F02_SA01_R02.csv', '../../Dataset/SA01/F02_SA01_R03.csv', '../../Dataset/SA01/F02_SA01_R04.csv', '../../Dataset/SA01/F02_SA01_R05.csv', '../../Dataset/SA01/F03_SA01_R01.csv', '../../Dataset/SA01/F03_SA01_R02.csv', '../../Dataset/SA01/F03_SA01_R03.csv', '../../Dataset/SA01/F03_SA01_R04.csv', '../../Dataset/SA01/F03_SA01_R05.csv', '../../Dataset/SA01/F04_SA01_R01.csv', '../../Dataset/SA01/F04_SA01_R02.csv', '../../Dataset/SA01/F04_SA01_R03.csv', '../../Dataset/SA01/F04_SA01_R04.csv', '../../Dataset/SA01/F04_SA01_R05.csv', '../../Dataset/SA01/F05_SA01_R01.csv', '../../Dataset/SA01/F05_SA01_R02.csv', '../../Dataset/SA01/F05_SA01_R03.csv', '../../Dataset/SA01/F05_SA01_R04.csv', '../../Dataset/SA01/F05_SA01_R05.csv', '../../Dataset/SA01/F06_

['../../Dataset/SA01/F01_SA01_R01.csv',
 '../../Dataset/SA02/F01_SA02_R01.csv',
 '../../Dataset/SA03/F01_SA03_R01.csv',
 '../../Dataset/SA04/F01_SA04_R01.csv',
 '../../Dataset/SA05/F01_SA05_R01.csv',
 '../../Dataset/SA06/F01_SA06_R01.csv',
 '../../Dataset/SA07/F01_SA07_R01.csv',
 '../../Dataset/SA08/F01_SA08_R01.csv',
 '../../Dataset/SA09/F01_SA09_R01.csv',
 '../../Dataset/SA10/F01_SA10_R01.csv',
 '../../Dataset/SA11/F01_SA11_R01.csv',
 '../../Dataset/SA12/F01_SA12_R01.csv',
 '../../Dataset/SA13/F01_SA13_R01.csv',
 '../../Dataset/SA14/F01_SA14_R01.csv',
 '../../Dataset/SA15/F01_SA15_R01.csv',
 '../../Dataset/SA16/F01_SA16_R01.csv',
 '../../Dataset/SA17/F01_SA17_R01.csv',
 '../../Dataset/SA18/F01_SA18_R01.csv',
 '../../Dataset/SA19/F01_SA19_R01.csv',
 '../../Dataset/SA20/F01_SA20_R01.csv',
 '../../Dataset/SA21/F01_SA21_R01.csv',
 '../../Dataset/SA22/F01_SA22_R01.csv',
 '../../Dataset/SA23/F01_SA23_R01.csv',
 '../../Dataset/SE06/F01_SE06_R01.csv',
 '../../Dataset/SA01/F01_SA01_R02.csv',


# Fall 데이터 전처리

In [7]:
fall_list = []

for path in total_fall_path:
    try:
        data = pd.read_csv(path, header=None)
        data = data.iloc[:, :3]
        data = np.abs(data)
                    
        max_value1 = data[0][1000:].max()
        max_value2 = data[1][1000:].max()
        max_value3 = data[2][1000:].max()
        
        max_idx1 = data[ data[0] == max_value1 ].index[0]
        max_idx2 = data[ data[1] == max_value2 ].index[0]
        max_idx3 = data[ data[2] == max_value3 ].index[0]
        
        min_idx = min(max_idx1, max_idx2, max_idx3)
        sliced_df = data[min_idx - 200 : min_idx - 40]
        
        col_1_lpf = butter_lowpass_filter(sliced_df[0].to_numpy(), cutoff, fs, order).astype('int64')
        col_2_lpf = butter_lowpass_filter(sliced_df[1].to_numpy(), cutoff, fs, order).astype('int64')
        col_3_lpf = butter_lowpass_filter(sliced_df[2].to_numpy(), cutoff, fs, order).astype('int64')
    
    
        col_1_std = np.sqrt(np.absolute(np.mean(sliced_df[0]) - sliced_df[0])**2 / len(sliced_df[0]))
        col_2_std = np.sqrt(np.absolute(np.mean(sliced_df[1]) - sliced_df[1])**2 / len(sliced_df[1]))
        col_3_std = np.sqrt(np.absolute(np.mean(sliced_df[2]) - sliced_df[2])**2 / len(sliced_df[2]))

        col_1 = col_1_std*col_1_std
        col_2 = col_2_std*col_2_std
        col_3 = col_3_std*col_3_std
        
        C_nine = np.sqrt(col_1 + col_2 + col_3).tolist()
        fall_list.append(C_nine)
        
    except:
        pass
    
fall_df = pd.DataFrame(fall_list)
print(fall_df.shape)
row, col = fall_df.shape


target_col = row * [1]
fall_df['target'] = target_col

print(fall_df.shape)
fall_df

(1799, 160)
(1799, 161)


,0,1,2,3,4,5,6,7,8,9,...,151,152,153,154,155,156,157,158,159,target
0,8.587570,8.082275,8.583630,8.305907,8.373136,8.637728,8.817040,8.833744,8.972186,8.881822,...,17.265368,15.013880,14.112880,15.146278,16.899204,16.299460,15.134409,14.433678,13.644165,1
1,1.999406,1.517492,1.027878,1.416907,1.164703,1.455368,2.198466,2.512338,2.653647,2.664020,...,5.121063,4.871245,5.053136,5.461882,5.141328,5.323735,5.405926,6.079458,6.960309,1
2,4.259701,4.488307,4.573700,4.452928,4.833968,4.898387,4.600184,5.124205,5.065259,5.044557,...,11.364513,11.139501,11.422257,11.195087,9.851371,10.709060,10.304155,11.023256,12.107987,1
3,2.669345,3.005478,2.828001,3.018279,3.599915,3.534607,3.602269,4.082787,3.983934,3.942013,...,5.133333,9.213204,13.020525,17.223968,19.715952,21.539403,31.394244,41.018313,45.606443,1
4,6.448080,6.260879,6.048599,5.535617,3.820270,3.638575,5.347992,6.860445,8.968252,10.258286,...,7.508121,9.431232,10.776983,12.342986,13.322400,13.224407,15.739987,17.515244,17.670540,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1794,4.838585,4.680523,4.863379,4.534765,4.590176,4.348656,4.370600,4.098670,4.365305,4.025712,...,9.186996,9.101727,9.463534,9.421661,9.591897,9.797749,9.966763,9.851687,9.293371,1
1795,6.934931,6.787648,6.058509,6.345322,6.518997,6.382157,6.680562,6.320062,6.336623,6.712487,...,16.212581,15.489265,15.462765,13.731470,14.585676,16.078229,17.864140,19.081191,19.071454,1
1796,5.681599,5.651233,5.505681,5.292236,5.316626,5.208064,5.442119,5.123485,5.323638,5.512644,...,14.453350,14.278046,14.446318,13.792404,13.433820,13.346097,13.129140,12.969511,12.320550,1
1797,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1


# Fall data 저장

In [8]:
final = pd.DataFrame(fall_df)
final = final.dropna(axis=0)
final.dropna()
final = final.astype(int)

final.to_csv('../result/f_c9_LPF_int.csv', index=False)

# All data 저장

In [9]:
adl = pd.read_csv('../result/a_c9_LPF_int.csv')
fall = pd.read_csv('../result/f_c9_LPF_int.csv')

fall_data = pd.DataFrame(fall)
print(fall_data.shape)
adl_data = pd.DataFrame(adl)
print(adl_data.shape)

file_list = [fall_data, adl_data]
result = pd.concat(file_list)
result.to_csv('../result/c9_LPF_int.csv', index=False)
print(result.shape)
result

(1768, 161)
(3108, 161)
(4876, 161)


,0,1,2,3,4,5,6,7,8,9,...,151,152,153,154,155,156,157,158,159,target
0,8,8,8,8,8,8,8,8,8,8,...,17,15,14,15,16,16,15,14,13,1
1,1,1,1,1,1,1,2,2,2,2,...,5,4,5,5,5,5,5,6,6,1
2,4,4,4,4,4,4,4,5,5,5,...,11,11,11,11,9,10,10,11,12,1
3,2,3,2,3,3,3,3,4,3,3,...,5,9,13,17,19,21,31,41,45,1
4,6,6,6,5,3,3,5,6,8,10,...,7,9,10,12,13,13,15,17,17,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3103,4,4,4,4,4,4,4,4,4,4,...,22,20,19,18,19,20,21,22,21,0
3104,0,0,0,0,0,0,0,0,0,0,...,3,3,4,5,6,7,8,9,10,0
3105,4,4,4,4,4,4,4,4,4,4,...,3,5,9,12,17,21,21,17,15,0
3106,3,2,2,2,2,2,2,2,2,2,...,17,16,15,15,15,16,16,18,19,0
